In [2]:
!pip install implicit

Dataset download Link: https://dev.to/amananandrai/language-model-implementation-bigram-model-22ij

use the events.csv file from the zip folder

In [3]:
#import libraries
import sys
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random



from sklearn.preprocessing import MinMaxScaler

import implicit
from datetime import datetime, timedelta


#Data Preprocessing
def create_data(datapath,start_date,end_date):
    df=pd.read_csv(datapath)
    df=df.assign(date=pd.Series(datetime.fromtimestamp(a/1000).date() for a in df.timestamp))
    df=df.sort_values(by='date').reset_index(drop=True) # for some reasons RetailRocket did NOT sort data by date
    df=df[(df.date>=datetime.strptime(start_date,'%Y-%m-%d').date())&(df.date<=datetime.strptime(end_date,'%Y-%m-%d').date())]
    df=df[['visitorid','itemid','event']]
    return df


#Download the kaggle RetailRocket data and give the events.csv file path
datapath= '/content/drive/MyDrive/sems 7 LAB/nlp/LAB/DATASET/exp 6/events.csv'
data=create_data(datapath,'2015-5-3','2015-5-18')
data['visitorid'] = data['visitorid'].astype("category")
data['itemid'] = data['itemid'].astype("category")
data['visitor_id'] = data['visitorid'].cat.codes
data['item_id'] = data['itemid'].cat.codes

data['event']=data['event'].astype('category')
data['event']=data['event'].cat.codes



sparse_item_user = sparse.csr_matrix((data['event'].astype(float), (data['item_id'], data['visitor_id'])))
sparse_user_item = sparse.csr_matrix((data['event'].astype(float), (data['visitor_id'], data['item_id'])))

#Building the model
model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=20)


alpha_val = 40
data_conf = (sparse_item_user * alpha_val).astype('double')


model.fit(data_conf)

  0%|          | 0/20 [00:00<?, ?it/s]

In [4]:
#Get Recommendations
user_id =   14
recommended = model.recommend(user_id, sparse_user_item[user_id])

recommended = pd.DataFrame(recommended)
recommended = recommended.T
recommended.columns = ['items', 'scores']

print(recommended)

      items        scores
0  167047.0  5.611404e-13
1   69511.0  4.969037e-13
2  118594.0  4.300158e-13
3  130076.0  3.755863e-13
4  153596.0  3.595782e-13
5   22659.0  3.504885e-13
6  124693.0  3.443873e-13
7   56781.0  3.431953e-13
8  165785.0  3.361322e-13
9  115528.0  3.356065e-13


In [5]:
#Get similar items
item_id = 7
n_similar = 4
similar = model.similar_items(item_id, n_similar)

similar = pd.DataFrame(similar)
similar = similar.T
similar.columns = ['items', 'scores']

print(similar)

      items    scores
0       7.0  1.000000
1  130467.0  0.999919
2   80629.0  0.999499
3  157686.0  0.999414
